以下のnotebookを参考に、モデリング前の時系列データの分析方法を学ぶ  

* [M5 Forecasting - Starter Data Exploration](https://www.kaggle.com/robikscube/m5-forecasting-starter-data-exploration)
* [Back to (predict) the future - Interactive M5 EDA](https://www.kaggle.com/headsortails/back-to-predict-the-future-interactive-m5-eda)

In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
from itertools import cycle
pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

## 1. ファイル内容の確認

In [ ]:
train = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')
prices = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')
calendar = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv')

sample_submit = pd.read_csv('../input/m5-forecasting-accuracy/sample_submission.csv')

In [ ]:
train_eval = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_evaluation.csv')
train_eval.tail()

### ・sales_train_validationファイル

In [ ]:
train.head(5)

項目
> + id: 商品id。item_idとstore_idの組み合わせ識別子
> + item_id：商品id2
> + dept_id：商品ジャンル1
> + cat_id：商品ジャンル2
> + d_1~d_1913：日別商品売上数  
  
データ分類
> + カテゴリデータ：dept_id, cat_id, store_id, state_id
> + 定量データ：d_1~d_1913

In [ ]:
train.shape

### ・sell_prices

In [ ]:
prices.head(5)

ストア毎の日別(週別？)の商品販売価格データ  
  
項目
> * store_id：ストア識別子
> * item_id：商品id
> * wm_yr_wk：販売日(週？)。calendarファイルとの連結キー。calenderファイルと組み合わせることで、日ごとの販売価格がわかる。
> * sell_price：販売価格  
  
データ分類
> * カテゴリデータ：store_id, item_id, wm_yr_wk
> * 定量データ：sell_price

In [ ]:
prices.shape

### ・Calendar

In [ ]:
calendar.head(5)

日別データ。販売日ごとの、wm_yr_wk(sell_priceとの連結キー)、曜日、年、日、イベント状況がわかる。  
  
項目
> + date:販売日
> + w_yr_wk:商品価格キー。sell_priceを参照することで販売価格わかる
> + weekday：曜日
> + wday：曜日表す数値
> + month：月
> + year：年
> + d：日。trainデータの日項目(d1~)に対応。
> + event_name_1, event_name_2：イベントあればイベント名
> + event_type_1, event_type_2：開催イベントの種類
> + snap_：国の食事支援対象かどうか

In [ ]:
calendar.shape

### 販売量予測対象日の確認  
  
+ train: 2011-01-29('d_1') ~ 2016-04-24('d_1913')
+ validation: 2016-04-25('d_1914') ~ 2016-05-22('d_1941')
+ evaluation: 2016-05-23('F1') ~ 2016-06-19('F28')  
  
約5年間のデータセットを用いて、販売量の予測モデルを作成し、2016年4月後半〜5月後半の予測でモデルの評価を行い、最終的な予測対象は、2016年5月後半〜6月前半

## 2.trainデータの分析

・trainデータとcalendarデータの年月日をマージして、日毎のデータ作成

In [ ]:
# 日別売上列取得用
d_cols = [c for c in train.columns if 'd_' in c]

date_train = train.set_index('id')[d_cols].T \
                  .merge(calendar.set_index('d')['date'],
                         left_index=True, right_index=True,
                         validate='1:1')
# 日付をインデックス化
date_train['date'] = pd.to_datetime(date_train['date'])
# インデックスを日付型に変更
date_train = date_train.set_index('date')

date_train.head()

### 2.1 商品ごとの売上状況  
  
20種類ほどランダムサンプリングして、時系列での販売状況を確認

In [ ]:
# calendarファイルとマージして、20商品の日別売上取得
twenty_examples = date_train.sample(20, random_state=8, axis=1)

twenty_examples.head()

In [ ]:
# 可視化１。商品20アイテムの販売状況比較
twenty_examples.plot(figsize=(15,5))
plt.legend(twenty_examples.columns, loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

> + 商品により販売量に大きな違いあり。
> + 販売量が0近辺で推移する商品もあれば、数十以上で買われる商品もあるので、商品により販売数量差が激しい。

In [ ]:
# 可視化２。商品ごとの販売推移
fig, axs = plt.subplots(10, 2, figsize=(20, 25))
axs = axs.flatten()
ax_idx = 0
for item in twenty_examples.columns:
    twenty_examples[item].plot(title=item,
                               color=next(color_cycle),
                               ax=axs[ax_idx])
    ax_idx += 1
plt.tight_layout()
plt.show()

> + 商品毎に、周期性がありそう。
> + ほとんどの商品で、一回の販売量が数個程度で、0も多い。
> + 初販売日が比較的新しい商品は、販売開始までの日別販売個数が0になっている。
> + 季節性がある商品は、買われるシーズン以外は、ほとんど0個。

### 2.2 商品全体の販売状況確認

In [ ]:
# 日別の全販売数計算
all_past_sales = date_train.copy()
all_past_sales['total'] = all_past_sales.sum(axis=1)

# 可視化
all_past_sales['total'].plot(figsize=(12, 3), alpha=0.8, title='Total Sales')
plt.show()

毎年末、0まで落ち込む日があるので何の日か確認

In [ ]:
# 売上最小日確認
all_past_sales.sort_values(by='total')[:5]

> + 毎年クリスマスは売上最小（年に1度の休みらしい）
> + 販売数は緩やかに上昇傾向
> + 一年の中で、季節的？な販売サイクルがありそう

In [ ]:
del all_past_sales

### 2.3 州ごとの販売状況  
  
州ごとに、月別販売数の推移を可視化

In [ ]:
import datetime

In [ ]:
# 州ごとに集計
for i in train['state_id'].unique():
    # 州ごとの商品取得
    state_col = [c for c in date_train.columns if i in c]
    # 

In [ ]:
# 州ごとに集計
for i in train['state_id'].unique():
    # 州ごとの商品取得
    state_col = [c for c in date_train.columns if i in c]
    # 月ごとに集計
    month_data = date_train.loc['2011-02-01':'2016-03-31' ,state_col].resample('M').sum()
    month_data['total'] = month_data.sum(axis=1)
    month_data['total'].plot(figsize=(12, 3), alpha=0.8, title='Monthly Sales Volume per State')
plt.legend(train['state_id'].unique())
plt.show()

> + カリフォルニアの売上が最も多い。ウィスコンシンは2015年後半にはテキサスを抜いた。
> + ３州の売上は異なるものの、上昇・下降の流れは、全て似通っており、季節性のサイクルがありそう。
> + 2015年の前半は、テキサス、ウィスコンシンにおいて、他の年とサイクルがずれており、小さく乱高下している。

### 2.4 カテゴリ別販売傾向  
  
カテゴリ別の月別売上の推移を確認

In [ ]:
# カテゴリごとに集計
for i in train['cat_id'].unique():
    # カテゴリごとの商品取得
    category_col = [c for c in date_train.columns if i in c]
    # 月ごとに集計
    month_data = date_train.loc['2011-02-01':'2016-03-31' ,category_col].resample('M').sum()
    month_data['total'] = month_data.sum(axis=1)
    month_data['total'].plot(figsize=(10, 3), alpha=0.8, title='Monthly Sales Volume per Category')
plt.legend(train['cat_id'].unique())
plt.show()

> + FOODSは、他２つよりはるかに販売量が大きいが、2012年後半から増加は一進一退。
> + FOODSは、一年の中で季節性の販売サイクルがありそう。
> + HOUSEHOLDは、年々緩やかに上昇しており、FOODSほどではないが、季節性のサイクルがありそう。
> + HOBBIESは、ほぼ水平に推移しており、季節性のサイクルもなさそう。

### 2.5 店舗別売上傾向  
  
店舗別の月別売上の推移を確認

In [ ]:
# store_idリスト
store_list = prices['store_id'].unique()
store_list

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15,5), sharex=True, sharey=True)
axes = axes.flatten()
ca_idx = 0
tx_idx = 1
wi_idx = 2
for s in store_list:
    store_items = [c for c in date_train.columns if s in c]
    month_data = date_train.loc['2011-02-01':'2016-03-31' ,store_items].resample('M').sum()
    month_data['total'] = month_data.sum(axis=1)
    if 'CA_' in s :
        sns.lineplot(y='total',x=month_data.index,
                     data=month_data,ax=axes[ca_idx], color=next(color_cycle))
    elif 'TX_' in s:
        sns.lineplot(y='total',x=month_data.index,
                     data=month_data,ax=axes[tx_idx], color=next(color_cycle))
    elif 'WI_' in s:
        sns.lineplot(y='total',x=month_data.index,
                     data=month_data,ax=axes[wi_idx], color=next(color_cycle))
        
axes[ca_idx].legend([c for c in store_list if 'CA_' in c])
axes[ca_idx].title.set_text('CA')
axes[tx_idx].legend([c for c in store_list if 'TX_' in c])
axes[tx_idx].title.set_text('TX')
axes[wi_idx].legend([c for c in store_list if 'WI_' in c])
axes[wi_idx].title.set_text('WI')
plt.show()

> + 州ごとに季節性のサイクルがありそう。
> + 全体で見ると、CA_3の販売量が他店舗より圧倒的に多い。（店舗サイズの違いか？）
> + 全般的に販売量は、年々増加している。
> + CAでは、CA_2の販売量が2015年から急激にあがっている。
> + TXでは、TX_3の販売量が2014年から伸び、TX_2に肉薄している。一方で、TX_2は、2013年後半に販売量を大きく減らし、その後回復していない。
> + WIでは、WI_1とWI_2の販売量が2012年後半に急激に増加している。（店舗を大きくしたか？）一方で、WI_3は、2012年から2015年まで減少傾向。

### 2.6 デパートメント毎の売上傾向  
  
デパートメント毎の月別販売量を可視化

In [ ]:
# デパートメントごとに集計
for i in train['dept_id'].unique():
    # デパートメントごとの商品取得
    dep_col = [c for c in date_train.columns if i in c]
    # 月ごとに集計
    month_data = date_train.loc['2011-02-01':'2016-03-31' ,dep_col].resample('M').sum()
    month_data['total'] = month_data.sum(axis=1)
    month_data['total'].plot(figsize=(10, 3), alpha=0.8, 
                             title='Monthly Sales Volume per Department',
                             color = next(color_cycle))
plt.legend(train['dept_id'].unique(),  loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

> + 全てのFOODSの販売量が大きいわけではなく、FOODS_3の販売量が非常に大きく、年内でも季節性のサイクルが見られる。
> + FOODS_1, FOODSL_2の販売量は、HOBBIES_1とほぼ変わらないが、FOODS_2は2015年から上昇傾向。
> + HOBBIES_2の販売量は、とても低く年が変わっても、変化なし。
> + HOSEHOLD_1は、緩やかな上昇傾向が続き、年内でもサイクルがありそう。

### 2.7 売上サイクルの確認  
  
曜日ごと、月ごとの販売状況の確認

・カテゴリごとの売上サイクル

In [ ]:
# 月ごと、曜日ごとの売上ヒートマップ作成用関数
# ----------------------------------------------------------------------------
# Author:  Nicolas P. Rougier
# License: BSD
# ----------------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from datetime import datetime
from dateutil.relativedelta import relativedelta


def calmap(ax, year, data):
    ax.tick_params('x', length=0, labelsize="medium", which='major')
    ax.tick_params('y', length=0, labelsize="x-small", which='major')

    # Month borders
    xticks, labels = [], []
    start = datetime(year,1,1).weekday()
    for month in range(1,13):
        first = datetime(year, month, 1)
        last = first + relativedelta(months=1, days=-1)

        y0 = first.weekday()
        y1 = last.weekday()
        x0 = (int(first.strftime("%j"))+start-1)//7
        x1 = (int(last.strftime("%j"))+start-1)//7

        P = [ (x0,   y0), (x0,    7),  (x1,   7),
              (x1,   y1+1), (x1+1,  y1+1), (x1+1, 0),
              (x0+1,  0), (x0+1,  y0) ]
        xticks.append(x0 +(x1-x0+1)/2)
        labels.append(first.strftime("%b"))
        poly = Polygon(P, edgecolor="black", facecolor="None",
                       linewidth=1, zorder=20, clip_on=False)
        ax.add_artist(poly)
    
    ax.set_xticks(xticks)
    ax.set_xticklabels(labels)
    ax.set_yticks(0.5 + np.arange(7))
    ax.set_yticklabels(["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"])
    ax.set_title("{}".format(year), weight="semibold")
    
    # Clearing first and last day from the data
    valid = datetime(year, 1, 1).weekday()
    data[:valid,0] = np.nan
    valid = datetime(year, 12, 31).weekday()
    # data[:,x1+1:] = np.nan
    data[valid+1:,x1] = np.nan

    # Showing data
    ax.imshow(data, extent=[0,53,0,7], zorder=10, vmin=-1, vmax=1,
              cmap="RdYlBu_r", origin="lower", alpha=.75)

In [ ]:
from sklearn.preprocessing import StandardScaler
sscale = StandardScaler()

In [ ]:
# from sklearn.preprocessing import StandardScaler
# sscale = StandardScaler()

# カテゴリ毎に、曜日毎、月毎の売上ヒートマップ
# 2013年から2015年まで1年毎に作成
for i in train['cat_id'].unique():
    fig, axes = plt.subplots(3, 1, figsize=(20, 8))
    items_col = [c for c in date_train.columns if i in c]
    
    # 2013年
    sale2013 = date_train.loc[date_train.index.isin(
                pd.date_range('31-Dec-2012', periods=371))][items_col].mean(axis=1)
     # 標準化の返り値は2次元配列なので、hstackで一次元配列化
    vals = np.hstack(sscale.fit_transform(sale2013.values.reshape(-1, 1)))
    # vals.reshape(53, 7)で、データを一行毎に一週間分にしている。
    calmap(axes[0], 2013, vals.reshape(53, 7).T)
    
    # 2014年
    sales2014 = date_train.loc[date_train.index.isin(
                pd.date_range('30-Dec-2013',periods=371))][items_col].mean(axis=1)
    vals = np.hstack(sscale.fit_transform(sales2014.values.reshape(-1, 1)))
    calmap(axes[1], 2014, vals.reshape(53,7).T)
    
    # 2015年
    sales2015 = date_train.loc[date_train.index.isin(
                pd.date_range('29-Dec-2014',periods=371))][items_col].mean(axis=1)
    vals = np.hstack(sscale.fit_transform(sales2015.values.reshape(-1, 1)))
    calmap(axes[2], 2015, vals.reshape(53,7).T)
    
    plt.suptitle(i, fontsize=30, x=0.4, y=1.01)
    plt.tight_layout()
    plt.show()


> + どのカテゴリも土日の販売量が多い。
> + hobby, householdは１月のweekdayの販売量が少ない。
> + Hobby：10月〜12月の販売量が多い。1月が一番少ない。
> + household：1月、12月は販売量少ない。
> + foods：12月の後半の週末は販売量少ない。あとは、通年で土日の販売量が多い。

・州ごとの、曜日別販売量サイクル

In [ ]:
# 曜日データ用意
week_train = train.set_index('id')[d_cols].T \
                    .merge(calendar.set_index('d')['wday'],
                          left_index=True, right_index=True,
                          validate='1:1')

# # 州ごとに、販売量を曜日でグループ化
for i in train['state_id'].unique():
    # 州ごとに商品取得
    state_col = [c for c in week_train.columns if i in c]
    # 曜日ごとにグループ化して平均値取得
    state_train = week_train[state_col + ['wday']].copy()
    state_train['total'] = state_train[state_col].sum(axis=1)
    state_week = state_train.groupby(by='wday')['total'].mean()
    # 曜日をインデックスにして可視化
    state_week.sort_index(inplace=True) 
    state_week = state_week.reset_index()
    week_dic = {1:'Sat', 2:'Sun', 3:'Mon', 4:'Tue', 5:'Wed', 6:'Thu', 7:'Fri'}
    state_week['weekday'] = state_week['wday'].map(week_dic)
    state_week.set_index('weekday', inplace=True)
    state_week['total'].plot(alpha=0.8,
                             title='Weekly Seasonality per State')
plt.legend(train['state_id'].unique(),  loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

> + WI州のみ、日曜日の販売量が土曜日より大きくない。（WI州の人たちは、日曜日よりも土曜日に買い物に行くのか？）
> + その他は、州ごとの週間購買行動に違いはない。

・州ごとの、月別販売量サイクル

In [ ]:
# 月データ用意
month_train = train.set_index('id')[d_cols].T \
                    .merge(calendar.set_index('d')['month'],
                          left_index=True, right_index=True,
                          validate='1:1')

# 州ごとに、販売量を月でグループ化
for i in train['state_id'].unique():
    # 州ごとに商品取得
    state_col = [c for c in month_train.columns if i in c]
    # 月ごとにグループ化して平均値取得
    state_train = month_train[state_col + ['month']].copy()
    state_train['total'] = state_train[state_col].sum(axis=1)
    state_month = state_train.groupby(by='month')['total'].mean()
    # 月をインデックスにして可視化
    state_month.sort_index(inplace=True) 
    state_month = state_month.reset_index()
    state_month.set_index('month', inplace=True)
    state_month['total'].plot(alpha=0.8,
                             title='Monthly Seasonality per State')
plt.legend(train['state_id'].unique(),  loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

> + 2月から4月(冬)にかけて、WI州のみ販売量が下がる。一方で、8月から12月(夏〜初冬)は緩やかに増加する。他の２州は、これと反対の動きをする。
> + 他は、大きな違いがない。

・州ごとカテゴリごとの曜日別販売量

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15,3))
axes = axes.flatten()
food_idx = 0
hobby_idx = 1
house_idx = 2

for s in train['state_id'].unique():
    # 州ごとに商品取得
    state_col = [c for c in week_train.columns if s in c]
    # カテゴリ毎に商品わける
    food_col = [c for c in state_col if 'FOODS_' in c]
    hobby_col = [c for c in state_col if 'HOBBIES_' in c]
    house_col = [c for c in state_col if 'HOUSEHOLD_' in c]
    
    for i in range(3):
        # food, hobby, house
        if i == 0:
            col = food_col
        elif i ==1:
            col = hobby_col
        else:
            col = house_col
            
        # 曜日ごとにグループ化して平均値取得
        state_train = week_train[col + ['wday']].copy()
        state_train['total'] = state_train[col].sum(axis=1)
        state_week = state_train.groupby(by='wday')['total'].mean()
        # 曜日をインデックスにして可視化
        state_week.sort_index(inplace=True) 
        state_week = state_week.reset_index()
        week_dic = {1:'Sat', 2:'Sun', 3:'Mon', 4:'Tue', 5:'Wed', 6:'Thu', 7:'Fri'}
        state_week['weekday'] = state_week['wday'].map(week_dic)
        state_week.set_index('weekday', inplace=True)
        
        if i == 0:
            state_week['total'].plot(ax = axes[food_idx])
        elif i == 1:
            state_week['total'].plot(ax = axes[hobby_idx])
        else:
            state_week['total'].plot(ax = axes[house_idx])


axes[food_idx].title.set_text('FOODS')
axes[hobby_idx].title.set_text('HOBBIES')
axes[house_idx].title.set_text('HOUSEHOLD')
axes[food_idx].legend([c for c in train['state_id'].unique()])
axes[hobby_idx].legend([c for c in train['state_id'].unique()])
axes[house_idx].legend([c for c in train['state_id'].unique()])
plt.show()

> + 曜日別では、カテゴリにかかわらず、州ごとの違いはそんなに無さそう。
> + FOODSの購入で、WI州のみ日曜日がトップではない。

・州ごとカテゴリごとの月別販売量

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15,3))
axes = axes.flatten()
food_idx = 0
hobby_idx = 1
house_idx = 2

for s in train['state_id'].unique():
    # 州ごとに商品取得
    state_col = [c for c in month_train.columns if s in c]
    # カテゴリ毎に商品わける
    food_col = [c for c in state_col if 'FOODS_' in c]
    hobby_col = [c for c in state_col if 'HOBBIES_' in c]
    house_col = [c for c in state_col if 'HOUSEHOLD_' in c]
    
    for i in range(3):
        # food, hobby, house
        if i == 0:
            col = food_col
        elif i ==1:
            col = hobby_col
        else:
            col = house_col

        # 月ごとにグループ化して平均値取得
        state_train = month_train[state_col + ['month']].copy()
        state_train['total'] = state_train[col].sum(axis=1)
        state_month = state_train.groupby(by='month')['total'].mean()
        # 月をインデックスにして可視化
        state_month.sort_index(inplace=True) 
        state_month = state_month.reset_index()
        state_month.set_index('month', inplace=True)
        
        if i == 0:
            state_month['total'].plot(ax = axes[food_idx])
        elif i == 1:
            state_month['total'].plot(ax = axes[hobby_idx])
        else:
            state_month['total'].plot(ax = axes[house_idx])


axes[food_idx].title.set_text('FOODS')
axes[hobby_idx].title.set_text('HOBBIES')
axes[house_idx].title.set_text('HOUSEHOLD')
axes[food_idx].legend([c for c in train['state_id'].unique()])
axes[hobby_idx].legend([c for c in train['state_id'].unique()])
axes[house_idx].legend([c for c in train['state_id'].unique()])
plt.show()

> + FOODS: WI州では、2〜4月(冬)に販売量が減少している。一方で、8月から微増傾向になる。他の２州は、これと反対の動き。
> + HOBBIES: ３州とも同じ動き。
> + HOUSEHOLD: CA州が、2〜4月に販売量が増加傾向。

## 3.CalendarとPriceデータの分析

### 3.1 Calendarデータの分析

In [ ]:
calendar.head()

・nan数の確認

In [ ]:
calendar.info()

> + event_name_2 と event_type_2 は、5レコードしかないので除外。
> + event_name_1, event_type_1, snap_を分析する。

・event_name_1, event_type_1

In [ ]:
print('イベントの種類（上位10種）：')
print(calendar['event_name_1'].value_counts()[:10])

In [ ]:
print('イベントタイプ：')
print(calendar['event_type_1'].value_counts())

In [ ]:
# イベントタイプの割合
event_type = calendar['event_type_1'].value_counts()
event_type = pd.DataFrame(index=event_type.index, data=event_type)

event_type['rate'] = round(event_type['event_type_1'] / event_type['event_type_1'].sum(), 2)
event_type.plot.bar(y='rate', ylim=[0, 0.5])

In [ ]:
# 全train日数に占めるイベント発生割合。
print('全train日すに占めるイベント発生割合：')
print(round(calendar['event_type_1'].notnull().sum() / len(calendar['date']), 2) * 100, ' %')

> + イベント発生率は、全日数の8%ほど。
> + イベントの内容は、バレンタインデーなどの宗教的な日や国の記念日に関わること。
> + イベント割合は、宗教イベントと国の記念日がそれぞれ3割超え、後は文化が2割でスポーツが1割
> + この手のイベントは全国で同じ日だと思われるので、州による違いは考慮する必要がないと考えられる。

・SNAP：生活保護制度（食料支援制度）

In [ ]:
CA = pd.DataFrame(data = calendar['snap_CA'].value_counts()).T
TX = pd.DataFrame(data = calendar['snap_TX'].value_counts()).T
WI = pd.DataFrame(data = calendar['snap_WI'].value_counts()).T
state_snap = pd.concat([CA, TX, WI])
state_snap['rate'] = round(state_snap[1] / state_snap.sum(axis=1), 2)
state_snap.plot.bar(y='rate', ylim=[0, 0.5])

> + 全ての州で同じだけsnap対象日がある。
> + snap対象日は、全日数の３割ほど。  
  
次に、州ごとのsnap対象日が同じか確認。

In [ ]:
# 日付をインデックス化
cal_in_data = calendar.copy()
cal_in_data['date'] = pd.to_datetime(cal_in_data['date'])
# インデックスを日付型に変更
cal_in_data = cal_in_data.set_index('date')

In [ ]:
snap_list = ['snap_CA', 'snap_TX', 'snap_WI']
fig, axes = plt.subplots(3, 1, figsize=(20, 8))

for i in range(len(snap_list)):
    snap2013 = cal_in_data.loc[cal_in_data.index.isin(
                    pd.date_range('31-Dec-2012', periods=371))][snap_list[i]]
    vals = np.hstack(sscale.fit_transform(snap2013.values.reshape(-1, 1)))
    # vals.reshape(53, 7)で、データを一行毎に一週間分にしている。
    calmap(axes[i], 2013, vals.reshape(53, 7).T)
    
    axes[i].set_title('2013: ' + snap_list[i])

plt.tight_layout()
plt.show()

> + 実施回数は同じでも、州により実施日が異なることがわかる。
> + ただし、どの州も、月の前半にsnap対象日が集中している。

・年度で実施日が異なるかをWI州で確認

### 3.2 Priceデータの分析

#### ・カテゴリごと、州ごとの販売価格の分布を確認  
  
(価格は、対数変換する）

In [ ]:
# 価格を対数変換
prices['log_price'] = np.log1p(prices['sell_price'])

・州ごと、カテゴリごとの分布

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20,5), sharex=True)
axs = axs.flatten()
axs_idx = 0
for i in train['cat_id'].unique():
    price_data = prices[prices['item_id'].str.contains(i)][['store_id', 'log_price']]
    for state in train['state_id'].unique():
        state_data = price_data[price_data['store_id'].str.contains(state)]['log_price']
        sns.distplot(state_data, ax=axs[axs_idx], kde=True, label=state)
    axs[axs_idx].set_title(i)
    axs[axs_idx].legend()
    axs_idx += 1

plt.show()

> + どの商品も州による販売価格の違いは無い。
> + FOODSの平均価格帯は、他のカテゴリより安い。
> + FOODSとHOUSEHOLDは山が１つなので、デパートメントの違いによる価格帯の波は無い。
> + HOBBIESは、山が２つあるので、価格帯が２種類あるっぽい。（デパートメントの違いの影響か？）

### 3.3 時系列分析

#### イベントの影響分析  
  
・カテゴリごとのイベント有りなしの購買行動の違い

In [ ]:
# イベント実施日取り出し
event_date = pd.to_datetime(calendar[calendar['event_type_1'].notnull()]['date'])
non_event_date = pd.to_datetime(calendar[calendar['event_type_1'].isnull()]['date'])

# イベント有り無しそれぞれの日別販売量取得
event_sales = date_train[date_train.index.isin(event_date)]
# クリスマスは閉店なので、クリスマス日のレコードは除外
event_sales = event_sales[~event_sales.index.isin(['2011-12-25','2012-12-25','2013-12-25','2014-12-25','2015-12-25'])]

non_event_sales = date_train[date_train.index.isin(non_event_date)]

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20,5), sharex=True)
axs = axs.flatten()
axs_idx = 0
for cat in train['cat_id'].unique():
    item_list = [c for c in non_event_sales.columns if cat in c]
    # 30日移動平均で販売量の推移確認
    event_data = event_sales[item_list].sum(axis=1).rolling(30).mean()
    non_event_data = non_event_sales[item_list].sum(axis=1).rolling(30).mean()
    sns.lineplot(event_data.index, event_data.values, ax= axs[axs_idx], label='event')
    sns.lineplot(non_event_data.index, non_event_data.values, ax= axs[axs_idx], label='non_event')
    axs[axs_idx].legend()
    axs[axs_idx].set_title(cat)
    axs_idx += 1
plt.suptitle('- 30 days moving average -', fontsize=20)
plt.show()

> + FOODSは、2013年以降、イベントによる違いがほとんど無い。
> + FOODS以外は、2013年以降、イベントを実施していない日の方が平均販売量が多い。
> + イベントの７割が宗教か国家的行事に関わることなので、そういった日は、あまり買い物しないのかもしれない。

・イベントの種類別の各カテゴリへの影響  
-> イベントの絶対数が少ないので一旦省略

・州ごとのイベント有りなしの購買行動の違い

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20,5), sharex=True)
axs = axs.flatten()
axs_idx = 0
for s in train['state_id'].unique():
    item_list = [c for c in non_event_sales.columns if s in c]
    # 30日移動平均で販売量の推移確認
    event_data = event_sales[item_list].sum(axis=1).rolling(30).mean()
    non_event_data = non_event_sales[item_list].sum(axis=1).rolling(30).mean()
    sns.lineplot(event_data.index, event_data.values, ax= axs[axs_idx], label='event')
    sns.lineplot(non_event_data.index, non_event_data.values, ax= axs[axs_idx], label='non_event')
    axs[axs_idx].legend()
    axs[axs_idx].set_title(s)
    axs_idx += 1
plt.suptitle('- 30 days moving average -', fontsize=20)
plt.show()

> + カテゴリ別の動きと同じように、2013年以降動きがトレンド化。
> + CAとWIは、2013年以降、イベント実施していない日の方が購買量多い。
> + TXは、2013年以降、イベントによる違いが殆ど無い。

#### SNAPの影響分析  
  
・州別のSNAP実施日の購買状況

In [ ]:
# イベント実施日取り出し
event_date = pd.to_datetime(calendar[calendar['event_type_1'].notnull()]['date'])
non_event_date = pd.to_datetime(calendar[calendar['event_type_1'].isnull()]['date'])

# イベント有り無しそれぞれの日別販売量取得
event_sales = date_train[date_train.index.isin(event_date)]
# クリスマスは閉店なので、クリスマス日のレコードは除外
event_sales = event_sales[~event_sales.index.isin(['2011-12-25','2012-12-25','2013-12-25','2014-12-25','2015-12-25'])]

non_event_sales = date_train[date_train.index.isin(non_event_date)]

In [ ]:
# snap実施日取り出し
snap_date = pd.to_datetime(calendar[(calendar['snap_CA'] == 1) |
                                    (calendar['snap_TX'] == 1) |
                                    (calendar['snap_TX'] == 1)]['date'])
                           
non_snap_date = pd.to_datetime(calendar[~((calendar['snap_CA'] == 1) |
                                         (calendar['snap_TX'] == 1) |
                                         (calendar['snap_TX'] == 1))]['date'])

# snap有り無しそれぞれの日別販売量取得
snap_sales = date_train[date_train.index.isin(snap_date)]
# クリスマスは閉店なので、クリスマス日のレコードは除外
snap_sales = snap_sales[~snap_sales.index.isin(['2011-12-25','2012-12-25','2013-12-25','2014-12-25','2015-12-25'])]

non_snap_sales = date_train[date_train.index.isin(non_snap_date)]

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20,5), sharex=True)
axs = axs.flatten()
axs_idx = 0
for s in train['state_id'].unique():
    item_list = [c for c in non_event_sales.columns if s in c]
    # 30日移動平均で販売量の推移確認
    snap_data = snap_sales[item_list].sum(axis=1).rolling(30).mean()
    non_snap_data = non_snap_sales[item_list].sum(axis=1).rolling(30).mean()
    sns.lineplot(snap_data.index, snap_data.values, ax= axs[axs_idx], label='snap')
    sns.lineplot(non_snap_data.index, non_snap_data.values, ax= axs[axs_idx], label='non_snap')
    axs[axs_idx].legend()
    axs[axs_idx].set_title(s)
    axs_idx += 1
plt.suptitle('- 30 days moving average of SNAP -', fontsize=20)
plt.show()

> + どの州でも、snap実施日の方が、販売数量が多くなっている。特にWI州は、snap未実施日との差が大きい。
> + ただし、snapは食事支援なので、snap実施日にすべてのカテゴリで販売量が増えているか確認が必要。

・州別、カテゴリ別のsnap別購買状況の確認

In [ ]:
# 州別、カテゴリ別のsnap別平均購買量の算出
day_num_list = []
non_day_num_list = []
s_c_list = []
for state in train['state_id'].unique():
    item_list = [c for c in non_event_sales.columns if state in c]
    for cat in train['cat_id'].unique():
        items = [s for s in item_list if cat in s]
        s_c_data = snap_sales[items]
        non_s_c_data = non_snap_sales[items]
    
        # snap有り無し別一日あたり平均販売数算出
        s_c_data['total'] = s_c_data.sum(axis=1)
        s_c_num = round(s_c_data['total'].sum() / len(s_c_data))
        non_s_c_data['total'] = non_s_c_data.sum(axis=1)
        non_s_c_num = round(non_s_c_data['total'].sum() / len(non_s_c_data))
        
        # リストに保存
        day_num_list.append(s_c_num)
        non_day_num_list.append(non_s_c_num)
        s_c_list.append(state + '_' + cat)

In [ ]:
# 可視化用にdf化
CA_df = pd.DataFrame({'category':['HOBBIES','HOBBIES', 'HOUSEHOLD','HOUSEHOLD', 'FOODS','FOODS'],
                      'snap': ['SNAP', 'no_SNAP', 'SNAP', 'no_SNAP', 'SNAP', 'no_SNAP'],
                      'num': [day_num_list[0], non_day_num_list[0],day_num_list[1], non_day_num_list[1],
                              day_num_list[2], non_day_num_list[2]]})

TX_df = pd.DataFrame({'category':['HOBBIES','HOBBIES', 'HOUSEHOLD','HOUSEHOLD', 'FOODS','FOODS'],
                      'snap': ['SNAP', 'no_SNAP', 'SNAP', 'no_SNAP', 'SNAP', 'no_SNAP'],
                      'num': [day_num_list[3], non_day_num_list[3],day_num_list[4], non_day_num_list[4],
                              day_num_list[5], non_day_num_list[5]]})
WI_df = pd.DataFrame({'category':['HOBBIES','HOBBIES', 'HOUSEHOLD','HOUSEHOLD', 'FOODS','FOODS'],
                      'snap': ['SNAP', 'no_SNAP', 'SNAP', 'no_SNAP', 'SNAP', 'no_SNAP'],
                      'num': [day_num_list[6], non_day_num_list[6],day_num_list[7], non_day_num_list[7],
                              day_num_list[8], non_day_num_list[8]]})

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15,4), sharex=True)
axs = axs.flatten()
df_list = [CA_df, TX_df, WI_df]
name = ['CA', 'TX', 'WI']
for i in range(3):
    sns.barplot(x='category', y='num', hue='snap', data= df_list[i], ax=axs[i])
    axs[i].set_title(name[i])

plt.suptitle('- daily Sales Volume per State -', fontsize=15)
plt.show()



> + どの州も傾向が変わらず、SNAP日はFOODSの購入量が伸びている。
> + 特に、WI州で、SNAP日にはFOODSの購入量の増加割合が大きい。
> + HOBBIES, HOUSEHOLDもわずかながらSNAP日は購入量が多い。（SNAP日に客足多いのか？）

・ヒートマップで、SNAP日の曜日別購買効果を確認

In [ ]:
# ヒートマップ用に、月別・曜日別販売量を集計

# snap日の曜日別、月別販売量計算
snap_sales['total'] = snap_sales.sum(axis=1)
m_w_snap = pd.merge(snap_sales, cal_in_data, left_index=True, right_index=True, how='left')
m_w_snap = m_w_snap[['total', 'wday', 'month']]
m_w_sum = m_w_snap.groupby(['month', 'wday']).sum() # 月別、曜日別販売数
m_w_count = m_w_snap.groupby(['month', 'wday']).count() # 月別、曜日別データ数
m_w_sum['day_num'] = round(m_w_sum['total'] / m_w_count['total'], 0) # 日別販売数算出
m_w_sum.drop('total', axis=1, inplace=True)

# 非snap日の曜日別、月別販売量計算
non_snap_sales['total'] = non_snap_sales.sum(axis=1)
non_m_w_snap = pd.merge(non_snap_sales, cal_in_data, left_index=True, right_index=True, how='left')
non_m_w_snap = non_m_w_snap[['total', 'wday', 'month']]
non_m_w_sum = non_m_w_snap.groupby(['month', 'wday']).sum()
non_m_w_count = non_m_w_snap.groupby(['month', 'wday']).count() # 月別、曜日別データ数
non_m_w_sum['day_num'] = round(non_m_w_sum['total'] / non_m_w_count['total'], 0) # 日別販売数算出
non_m_w_sum.drop('total', axis=1, inplace=True)

# 各集計を合体して、snap実施による一日あたり販売量の増加率を算出
merged_m_w = pd.merge(m_w_sum, non_m_w_sum, left_index=True, right_index=True, suffixes=('_SNAP', '_non'))
merged_m_w['rate'] = round((merged_m_w['day_num_SNAP'] / merged_m_w['day_num_non']), 2)

In [ ]:
# heatmap用df
heat_m_w = pd.DataFrame(data=merged_m_w['rate'].values.reshape(12,7).T,
                        index=['Sat', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri'],
                        columns=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])

# heatmap
fig = plt.figure(figsize=(10,5))
sns.heatmap(heat_m_w, alpha=0.9, cmap='Reds')

> + ヒートマップを見ると、年間を通して平日の月〜木の色が濃く、SNAPがある平日は、購買量が1〜2割増加している。
> + SNAPの影響は、FOODSが大きかったので、この増加のほとんどはFOODSと考えられる。

## 4. 個別分析  
  
ランダムに、FOODS, HOBBIES, HOUSEHOLDから商品を１つづつ選択し、1年間での売上推移や、SNAPやイベントの影響、価格変動などを確認

### 4.1 個別商品の、一年間の販売における、SNAP、イベントの影響分析  
  
ランダムに、FOODS, HOBBIES, HOUSEHOLDから商品を１つづつ選択し分析する

In [ ]:
# 各カテゴリよりサンプルitem抽出
food_sample = date_train.loc[:,date_train.columns.str.contains('FOODS')] \
                        .sample(1, random_state=7, axis=1)
house_sample = date_train.loc[:,date_train.columns.str.contains('HOUSEHOLD')] \
                        .sample(1, random_state=7, axis=1)
hobby_sample = date_train.loc[:,date_train.columns.str.contains('HOBBIES')] \
                        .sample(1, random_state=7, axis=1)

# 2015年のデータ抽出
food_data = food_sample.loc['2015-01-01':'2015-12-31',:][food_sample.columns]
house_data = house_sample.loc['2015-01-01':'2015-12-31',:][house_sample.columns]
hobby_data = hobby_sample.loc['2015-01-01':'2015-12-31',:][hobby_sample.columns]

# 各商品の2015年のsnap日を取得
snap_food_2015 = calendar[(calendar['date'].str.contains('2015-')) 
                        & (calendar['snap_WI'] == 1) ]['date']
snap_house_2015 = calendar[(calendar['date'].str.contains('2015-')) 
                        & (calendar['snap_TX'] == 1) ]['date']
snap_hobby_2015 = calendar[(calendar['date'].str.contains('2015-')) 
                        & (calendar['snap_CA'] == 1) ]['date']
# 2015年のイベント日を取得
event_2015 = calendar[(calendar['date'].str.contains('2015-')) 
                        & (calendar['event_type_1'].notnull()) ]['date']

# snap日の各売上データ抽出
snap_food_data = food_data[food_data.index.isin(snap_food_2015)]
snap_house_data = house_data[house_data.index.isin(snap_house_2015)]
snap_hobby_data = hobby_data[hobby_data.index.isin(snap_hobby_2015)]

# event日の各売上データ抽出
event_food_data = food_data[food_data.index.isin(event_2015)]
event_house_data = house_data[house_data.index.isin(event_2015)]
event_hobby_data = hobby_data[hobby_data.index.isin(event_2015)]

In [ ]:
# 2015年での、３アイテムの販売量と、イベント日、snap日の可視化

y_list = [food_data, house_data, hobby_data]
snap_y_list = [snap_food_data, snap_house_data, snap_hobby_data]
event_y_list = [event_food_data, event_house_data, event_hobby_data]

fig, axs = plt.subplots(3, 1, figsize=(20,8), sharex=True)
axs = axs.flatten()

for i in range(3):
    sns.lineplot(y_list[i].index, y_list[i].values.flatten(), alpha=0.4, ax=axs[i], color=next(color_cycle))
    sns.scatterplot(event_y_list[i].index, event_y_list[i].values.flatten(),
                    marker='o', color='red', ax=axs[i], label='event_day')
    sns.scatterplot(snap_y_list[i].index, snap_y_list[i].values.flatten(),
                    marker='+', color='black', ax=axs[i], label='snap_day')
    axs[i].set_title('2015: ' + y_list[i].columns[0])

plt.show()

> + サンプルで選んだ３品目のうち、event日に食料が他のカテゴリより多く買われている感じだが、snapに関しては３サンプルで大きな違いがあるように見えない。
> + 単品だとズレがあるので、もう何アイテムか選んだ方がいいのかもしれない。

・販売価格の変化と販売量の変化の分析  
  
ー＞インフレ率とかも考えなければいけなくなるので、一旦スキップ

## 5. その他  
  

・ゼロ値の分布確認  
すべての商品が、毎日買われているわけではないので、過去5年のデータで、どれだけゼロ値があるのか確認。　　
2011-01-29から2016-02-04までの日々の購買数0の商品割合を算出し、購買数0の日毎割合を調べる。

In [ ]:
# 日毎の販売数0の商品割合算出
count_0 = lambda x: x.value_counts()[0]
date_train['zero_num'] = date_train.apply(count_0, axis=1)
date_train['zero_ratio'] = round((date_train['zero_num'] / len(date_train.columns)), 2)

# ヒストグラム
sns.distplot(date_train['zero_ratio'], kde=True)

> + ほとんどの日別レコードで、7割がゼロ値と。ゼロだらけのデータとなっている。  
  
次に、年ごとの0値の分布状況を確認

In [ ]:
# 年ごとのデータ取り出し
y_2011 = date_train['2011-01-01':'2011-12-31'][['zero_num','zero_ratio']]
y_2012 = date_train['2012-01-01':'2012-12-31'][['zero_num','zero_ratio']]
y_2013 = date_train['2013-01-01':'2013-12-31'][['zero_num','zero_ratio']]
y_2014 = date_train['2014-01-01':'2014-12-31'][['zero_num','zero_ratio']]
y_2015 = date_train['2015-01-01':'2015-12-31'][['zero_num','zero_ratio']]
y_2016 = date_train['2015-01-01':'2015-12-31'][['zero_num','zero_ratio']]

# 年ごとに、日別0値割合計算
item_num = len(date_train.columns) - 2 # 商品数
y_2011['zero_ratio'] = round((y_2011['zero_num'] / item_num), 2)
y_2012['zero_ratio'] = round((y_2012['zero_num'] / item_num), 2)
y_2013['zero_ratio'] = round((y_2013['zero_num'] / item_num), 2)
y_2014['zero_ratio'] = round((y_2014['zero_num'] / item_num), 2)
y_2015['zero_ratio'] = round((y_2015['zero_num'] / item_num), 2)
y_2016['zero_ratio'] = round((y_2016['zero_num'] / item_num), 2)

In [ ]:
# 日別0値割合を年ごとにヒストグラムで可視化

fig, axs = plt.subplots(1, 6, figsize=(20,3), sharex=True)
axs = axs.flatten()

year_list = [y_2011, y_2012, y_2013, y_2014, y_2015, y_2016]
year_name = [2011, 2012, 2013, 2014, 2015, 2016]

for i in range(6):
    sns.distplot(year_list[i]['zero_ratio'], kde=True, ax=axs[i])
    axs[i].set_title(year_name[i])
plt.suptitle('- zero_ratio distribution per day -', fontsize=14, y=1.2)
plt.show()

> + 2011年、2012年に関しては、ほとんどの日毎レコードで0値が7割・8割という0だらけのデータ。
> + 2013年から、0値が減少している。  
  
ー＞2011年、2012年のレコードは使い物にならないか？

以上見てきた各分析結果を、時系列予測モデルに反映させなければならない・・・。